Creación de containers

Antes de poder utilizar Amazon SageMaker para crear procesos que nos permitan preparar nuestros datos, entrenar un modelo, optimizar algún algoritmo y desplegar el modelo como un Endpoint, primero vamos a crear 3 contenedores Docker: uno para la preparación de los datos, otro para el entrenamiento de los modelos y el último para el despliegue y hosting del modelo.

Amazon SageMaker proporciona contenedores Docker con los frameworks más comúnmente utilizados hoy en día, así como contenedores con implementaciones de algoritmos comunes y si bien podríamos simplemente utilizar alguno de esos contenedores, en este capítulo crearemos nuestros propios contenedores con las dependencias que pudiéramos llegar a requerir.

necesitamos autenticarnos y para esto debemos obtener una cuenta en DockerHub y sustituir usuario y constraseña por los valores correspondientes.

In [ ]:
secret_name = 'dockerhub'
sagemaker_utils.create_secret(secret_name,'usuario','contraseña')


Necesitaremos un rol de ejecución para ser utilizado en el proyecto de AWS Code Build. Si estamos ejecutando el Notebook con permisos suficientes para crear un rol de IAM, podemos crear el rol simplemente ejecutando el siguiente método, de lo contrario tendría que ser creado de forma manual.

In [ ]:
policy_document={
        "Version": "2022-05-06",
        "Statement": [               
            {
                "Effect": "Allow",
                "Action": [
                    "ecr:BatchCheckLayerAvailability",
                    "ecr:CompleteLayerUpload",
                    "ecr:GetAuthorizationToken",
                    "ecr:InitiateLayerUpload",
                    "ecr:PutImage",
                    "ecr:UploadLayerPart",
                    "ecr:BatchGetImage",
                    "ecr:GetDownloadUrlForLayer",
                    "logs:CreateLogGroup",
                    "logs:CreateLogStream",
                    "logs:PutLogEvents",
                    "s3:PutObject",
                    "s3:GetObject",
                    "s3:GetObjectVersion",
                    "secretsmanager:GetSecretValue"
                ],
                "Resource": "*"
            }
        ]
    }

codebuild_role = sagemaker_utils.create_codebuild_execution_role('CodeBuildExecutionRole', policy_document)


Especificamos las dependencias requeridas para cada uno de los contenedores Docker que crearemos.

In [ ]:
docker_images = {'Processing':{'libraries':{'pandas':'1.4.0',
                                            'numpy':'1.22.3',
                                            'scikit-learn':'1.0.2'}},
                 'Training':{'libraries':{'pandas':'1.4.0',
                                            'numpy':'1.22.3',
                                            'scikit-learn':'1.0.2',
                                          'sagemaker-training':'3.9.2'}},
                 'Inference':{'libraries':{'pandas':'1.4.0',
                                            'numpy':'1.22.3',
                                            'scikit-learn':'1.0.2',
                                           'multi-model-server':'1.1.8',                            
                                           'sagemaker-inference':'1.5.11',
                                           'boto3':'1.21.43',
                                           'itsdangerous':'2.0.1'},
                              'dependencies':[('serving','/opt/ml/serving')],
                              'others':['RUN pip install -e /opt/ml/serving',
                                        'LABEL com.amazonaws.sagemaker.capabilities.multi-models=false',
                                        'LABEL com.amazonaws.sagemaker.capabilities.accept-bind-to-port=true'],
                              'entrypoint':['python','/opt/ml/serving/custom_inference/serving.py'],
                              'cmd':['serve']}}


Creamos y publicamos las imágenes Docker en Amazon Elastic Container Registry para posteriormente poder ser utilizados en los jobs que crearemos y lanzaremos

In [ ]:
for image in docker_images:
    parameters = {'image_name': f'{prefix}-{image.lower()}',
                  'base_image': 'python:3.7.6-slim-buster',
                  's3_path': f's3://{bucket}/{images_directory}',
                  'role': codebuild_role,  
                  'secret': secret_name,
                  'wait': False}
    
    parameters.update(docker_images[image])
    
    docker_images[image]['build_id'] = sagemaker_utils.create_docker_image(**parameters)  


In [ ]:
#definimos un waiter

image_uris = sagemaker_utils.wait_for_build([docker_images[image]['build_id'] for image in docker_images])
for image in docker_images:
    docker_images[image]['image_uri'] = image_uris[docker_images[image]['build_id']]
